In [24]:
import xml.etree.ElementTree as ET
from collections import OrderedDict, namedtuple

class Corporation:
  def __init__(self, code, name, stock_code):
    self.code = code
    self.name = name
    self.stock_code = stock_code

corp_info = namedtuple('corp_info', ['code', 'name', 'stock_code'])

def getCorporationInfo(element):
  code = element.find('corp_code').text
  name = element.find('corp_name').text
  stock_code = element.find('stock_code').text 
  return corp_info(code, name, stock_code)

def getAllCorporation(treePath):
  odict = OrderedDict()
  tree = ET.parse(treePath)
  for node in tree.findall("./list"):
    info = getCorporationInfo(node)
    odict[info.name] = Corporation(info.code, info.name, info.stock_code)
  return odict

def getAllStockCorporation(treePath):
  odict = OrderedDict()
  tree = ET.parse(treePath)
  for node in tree.findall("./list"):
    info = getCorporationInfo(node)
    if info.stock_code != " ":
      odict[info.name] = Corporation(info.code, info.name, info.stock_code)
  return odict

def getCodeNameDict(treePath):
  odict = OrderedDict()
  tree = ET.parse(treePath)
  for node in tree.findall("./list"):
    info = getCorporationInfo(node)
    odict[info.code] = info.name
  return odict

def getStockCodeNameDict(treePath):
  odict = OrderedDict()
  tree = ET.parse(treePath)
  for node in tree.findall("./list"):
    info = getCorporationInfo(node)
    if info.stock_code != " ":
      odict[info.code] = info.name
  return odict

stock_corps = getAllStockCorporation("data\CORPCODE.xml")
stock_code_dict = getStockCodeNameDict("data/CORPCODE.xml")
# print(stock_code_dict.items(), "im dict")

In [25]:
import requests
from enum import Enum
import json
from typing import Iterable

CRTF_KEY = "9315957629f592ce316b16f14bfe9897166091f2"

class ReportCode(Enum):
  first_quarter = "11013"
  second_quarter = "11012"
  third_quarter = "11014"
  fourth_quarter = "11011"

SINGLE_CORP_ACCOUNT_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcnt.json"
MULTI_CORP_ACCOUNT_URL = "https://opendart.fss.or.kr/api/fnlttMultiAcnt.json"
def getCorpAccount(corp_code: str, year: str, quarter: ReportCode):
  res = requests.get(SINGLE_CORP_ACCOUNT_URL, params={
    "crtfc_key": CRTF_KEY,
    "corp_code": corp_code,
    "bsns_year": year,
    "reprt_code": quarter.value
  })
  res.encoding = 'utf-8'
  obj = json.loads(res.text)
  return obj

def getMultiCorpAccount(corp_code: Iterable[str], year: str, quarter: ReportCode):
  corps_str = ",".join(corp_code) if isinstance(corp_code, Iterable) else corp_code
  res = requests.get(MULTI_CORP_ACCOUNT_URL, params={
   "crtfc_key": CRTF_KEY,
   "corp_code": corps_str,
   "bsns_year": year,
   "reprt_code": quarter.value 
  })
  res.encoding = 'utf-8'
  obj = json.loads(res.text)
  return obj

# data = getCorpAccount("00899459", "2019", ReportCode.first_quarter.value)
# print(data["message"])

# multi_data = getMultiCorpAccount(["00899459", "00126380"], "2019", ReportCode.first_quarter.value)
# print(multi_data)

In [26]:
md_data = getMultiCorpAccount(["00899459", "00126380"], "2019", ReportCode.first_quarter)
# print(md_data)

In [27]:
from itertools import islice

corporation_account_info = namedtuple('corp_acc_info', ('rcept_no', 'bsns_year', 'stock_code', 'reprt_code'))

def makeStockAccountJson(stock_corps, year: str, quarter: ReportCode):
  stock_corps_copy = stock_corps.copy()
  corporation_json_result = {}
  corporation_code_stack = []

  while len(stock_corps_copy) > 1:
    corporation = stock_corps_copy.popitem()[1]
    corporation_code_stack.append(corporation.code)

    if len(corporation_code_stack) == 100 or len(stock_corps_copy) == 0:
      multi_data = getMultiCorpAccount(corporation_code_stack, year, quarter)
      corporation_code_stack = []
      if (multi_data["status"]) == "000":
        for data in multi_data["list"]:
          try:
            name = stock_code_dict[data["corp_code"]]
            corporation_json_result[name].append(data)
          except:
            corporation_json_result[name] = []
            corporation_json_result[name].append(data)
      else:
        print("주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나",
        "이상이 있을 수 있으니 확인해주세요.\n",
        "------------------------------------\n",
        f'상태 코드: {multi_data["status"]}, {multi_data["message"]}\n',
        f'현재 회사: {corporation.name}, 남은 회사 수: {len(stock_corps_copy)}\n',
        f'스택: {corporation_code_stack}, 스택 길이: {len(corporation_code_stack)}\n'
        )

  with open(f'../src/data/json/{year}_{quarter.name}_정보.json', 'w', encoding='utf-8') as outfile:
    json.dump(corporation_json_result, outfile, indent=4,ensure_ascii=False)


In [28]:
years = ('2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022')
quarters = (ReportCode.first_quarter, ReportCode.second_quarter, ReportCode.third_quarter, ReportCode.fourth_quarter)

for year in years:
  for quarter in quarters:
    makeStockAccountJson(stock_corps, year, quarter)

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 회사: SDN, 남은 회사 수: 3314
 스택: [], 스택 길이: 0

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 회사: 위니아에이드, 남은 회사 수: 3214
 스택: [], 스택 길이: 0

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 회사: 서산, 남은 회사 수: 3114
 스택: [], 스택 길이: 0

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 회사: 유진스팩8호, 남은 회사 수: 3014
 스택: [], 스택 길이: 0

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 회사: 톱텍, 남은 회사 수: 2914
 스택: [], 스택 길이: 0

주의: 데이터를 제대로 수신하지 못했습니다. 작동에는 별 문제가 없으나 이상이 있을 수 있으니 확인해주세요.
 ------------------------------------
 상태 코드: 013, 조회된 데이타가 없습니다.
 현재 

In [29]:
print(ReportCode.first_quarter.value)

11013
